# LSTM Classification with MPQA Dataset
<hr>

We will build a text classification model using LSTM model on the MPQA Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 

## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/MPQA/MPQA.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10606, 3)


,sentence,label,split
0,complaining,0,train
1,failing to support,0,train
2,desperately needs,0,train
3,many years of decay,0,train
4,no quick fix,0,train
...,...,...,...
10601,urged,1,train
10602,strictly abide,1,train
10603,hope,1,train
10604,strictly abide,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10606 entries, 0 to 10605
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10606 non-null  object
 1   label     10606 non-null  int32 
 2   split     10606 non-null  object
dtypes: int32(1), object(2)
memory usage: 207.3+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,7294,7294
1,3312,3312


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'complaining'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  no quick fix
Into a sequence of int: [25, 945, 1476]
Into a padded sequence: [  25  945 1476    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
of 3
to 4
a 5
and 6
not 7
is 8
in 9
be 10
6236


# Model 1: Embedding Random
<hr>

<img src="model.png" style="width:700px;height:400px;"> <br>

## LSTM Model

In [10]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.LSTM(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [11]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 487,009
Trainable params: 487,009
Non-trainable params: 0
_________________________________________________________________


In [12]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=5, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [13]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1

    # Define the input shape
    model = define_model(input_dim=vocab_size, max_length=max_len)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=15, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record = record.append(temp, ignore_index=True)
print()
print(record)
print()

Training 1: 
Epoch 1/15
299/299 [==============================] - 29s 67ms/step - loss: 0.5662 - accuracy: 0.7385 - val_loss: 0.3534 - val_accuracy: 0.8633
Epoch 2/15
299/299 [==============================] - 16s 54ms/step - loss: 0.2214 - accuracy: 0.9263 - val_loss: 0.3910 - val_accuracy: 0.8596
Epoch 3/15
299/299 [==============================] - 16s 54ms/step - loss: 0.1346 - accuracy: 0.9540 - val_loss: 0.4198 - val_accuracy: 0.8520
Epoch 4/15
299/299 [==============================] - 16s 54ms/step - loss: 0.1008 - accuracy: 0.9631 - val_loss: 0.4683 - val_accuracy: 0.8549
Epoch 5/15
299/299 [==============================] - 16s 54ms/step - loss: 0.0902 - accuracy: 0.9663 - val_loss: 0.5558 - val_accuracy: 0.8473
Epoch 6/15
299/299 [==============================] - 16s 54ms/step - loss: 0.0783 - accuracy: 0.9686 - val_loss: 0.5774 - val_accuracy: 0.8483
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
Test Accuracy: 86.33365035057068
Traini

## Summary

In [14]:
record

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,86.33365,83.317626,85.485393,84.637135,88.124412,86.804903,86.415094,87.358493,86.792451,87.452829,86.272199


In [15]:
report = record
report = report.to_excel('LSTM_MPQA.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [11]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [12]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [13]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [14]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 6083 words present from 6236 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [15]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()
print('emb_mean: ', emb_mean)
print('emb_std: ', emb_std)

emb_mean:  -0.003527845
emb_std:  0.13315111


In [16]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [17]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## LSTM Model

In [18]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_2(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = False),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.LSTM(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [19]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               186880    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 487,009
Trainable params: 187,009
Non-trainable params: 300,000
_________________________________________________________________


## Train and Test the Model

In [20]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') >= 0.9):
            print("\nReached 90% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [21]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
emb_mean = emb_mean
emb_std = emb_std
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

    # Define the input shape
    model = define_model_2(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=100, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record2 = record2.append(temp, ignore_index=True)
print()
print(record2)
print()

Training 1: 
Epoch 1/100
299/299 [==============================] - 23s 49ms/step - loss: 0.4740 - accuracy: 0.7748 - val_loss: 0.3101 - val_accuracy: 0.8775
Epoch 2/100
299/299 [==============================] - 11s 36ms/step - loss: 0.2814 - accuracy: 0.8904 - val_loss: 0.3037 - val_accuracy: 0.8784
Epoch 3/100
299/299 [==============================] - 11s 37ms/step - loss: 0.2606 - accuracy: 0.9024 - val_loss: 0.2937 - val_accuracy: 0.8803
Epoch 4/100
299/299 [==============================] - 11s 36ms/step - loss: 0.2294 - accuracy: 0.9115 - val_loss: 0.2953 - val_accuracy: 0.8803
Epoch 5/100
299/299 [==============================] - 11s 37ms/step - loss: 0.2249 - accuracy: 0.9163 - val_loss: 0.2936 - val_accuracy: 0.8822
Epoch 6/100
299/299 [==============================] - 11s 37ms/step - loss: 0.2125 - accuracy: 0.9198 - val_loss: 0.3086 - val_accuracy: 0.8709
Epoch 7/100
299/299 [==============================] - 11s 37ms/step - loss: 0.1931 - accuracy: 0.9268 - val_loss: 0.

299/299 [==============================] - 17s 57ms/step - loss: 0.1530 - accuracy: 0.9457 - val_loss: 0.3204 - val_accuracy: 0.8746
Epoch 10/100
299/299 [==============================] - 17s 57ms/step - loss: 0.1449 - accuracy: 0.9471 - val_loss: 0.3369 - val_accuracy: 0.8728
Epoch 11/100
299/299 [==============================] - 17s 57ms/step - loss: 0.1306 - accuracy: 0.9524 - val_loss: 0.3436 - val_accuracy: 0.8803
Epoch 12/100
299/299 [==============================] - 17s 57ms/step - loss: 0.1247 - accuracy: 0.9535 - val_loss: 0.3787 - val_accuracy: 0.8794
Epoch 13/100
299/299 [==============================] - 18s 59ms/step - loss: 0.1090 - accuracy: 0.9625 - val_loss: 0.3725 - val_accuracy: 0.8822
Epoch 14/100
299/299 [==============================] - 18s 59ms/step - loss: 0.1050 - accuracy: 0.9624 - val_loss: 0.4018 - val_accuracy: 0.8784
Epoch 15/100
299/299 [==============================] - 18s 59ms/step - loss: 0.1030 - accuracy: 0.9596 - val_loss: 0.4103 - val_accuracy

Test Accuracy: 88.87841701507568
Training 7: 
Epoch 1/100
299/299 [==============================] - 43s 116ms/step - loss: 0.4633 - accuracy: 0.7826 - val_loss: 0.3220 - val_accuracy: 0.8764
Epoch 2/100
299/299 [==============================] - 33s 111ms/step - loss: 0.2904 - accuracy: 0.8840 - val_loss: 0.3029 - val_accuracy: 0.8925
Epoch 3/100
299/299 [==============================] - 32s 108ms/step - loss: 0.2492 - accuracy: 0.9004 - val_loss: 0.3103 - val_accuracy: 0.8849
Epoch 4/100
299/299 [==============================] - 32s 108ms/step - loss: 0.2462 - accuracy: 0.9063 - val_loss: 0.2915 - val_accuracy: 0.8953
Epoch 5/100
299/299 [==============================] - 32s 108ms/step - loss: 0.2233 - accuracy: 0.9138 - val_loss: 0.2938 - val_accuracy: 0.8934
Epoch 6/100
299/299 [==============================] - 31s 105ms/step - loss: 0.2105 - accuracy: 0.9189 - val_loss: 0.2994 - val_accuracy: 0.8906
Epoch 7/100
299/299 [==============================] - 31s 105ms/step - loss: 

## Summary

In [22]:
record2

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,88.595665,88.784164,89.443922,88.407165,88.030159,88.878417,89.811319,87.26415,86.98113,85.849059,88.204515


In [23]:
report = record2
report = report.to_excel('LSTM_MPQA_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## LSTM Model

In [24]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model_3(input_dim = None, output_dim=300, max_length = None, emb_matrix=None):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, ),
                                  # Assign the embedding weight with word2vec embedding marix
                                  weights = [emb_matrix],
                                  # Set the weight to be not trainable (static)
                                  trainable = True),
        
#         tf.keras.layers.LSTM(units=128, return_sequences=True),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional((tf.keras.layers.LSTM(64))),
#         tf.keras.layers.Dropout(0.5),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        # Propagate X through a Dense layer with 1 unit
        tf.keras.layers.Dense(units=1, activation='sigmoid')
    ])
    
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [25]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 300)          300000    
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               186880    
_________________________________________________________________
dropout_11 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 487,009
Trainable params: 487,009
Non-trainable params: 0
_________________________________________________________________


## Train and Test the Model

In [26]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=8, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [27]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
emb_mean = emb_mean
emb_std = emb_std
columns = ['acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp=0

# kfold.split() will return set indices for each split
acc_list = []
for train, test in kfold.split(sentences):
    
    exp+=1
    print('Training {}: '.format(exp))
    
    train_x, test_x = [], []
    train_y, test_y = [], []

    for i in train:
        train_x.append(sentences[i])
        train_y.append(labels[i])

    for i in test:
        test_x.append(sentences[i])
        test_y.append(labels[i])

    # Turn the labels into a numpy array
    train_y = np.array(train_y)
    test_y = np.array(test_y)

    # encode data using
    # Cleaning and Tokenization
    tokenizer = Tokenizer(oov_token=oov_tok)
    tokenizer.fit_on_texts(train_x)

    # Turn the text into sequence
    training_sequences = tokenizer.texts_to_sequences(train_x)
    test_sequences = tokenizer.texts_to_sequences(test_x)

    max_len = max_length(training_sequences)

    # Pad the sequence to have the same size
    Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
    Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)

    # Define the input shape
    model = define_model_3(input_dim=vocab_size, max_length=max_len, emb_matrix=emb_matrix)

    # Train the model
    model.fit(Xtrain, train_y, batch_size=32, epochs=100, verbose=1, 
              callbacks=[callbacks], validation_data=(Xtest, test_y))

    # evaluate the model
    loss, acc = model.evaluate(Xtest, test_y, verbose=0)
    print('Test Accuracy: {}'.format(acc*100))

    acc_list.append(acc*100)

mean_acc = np.array(acc_list).mean()
entries = acc_list + [mean_acc]

temp = pd.DataFrame([entries], columns=columns)
record3 = record3.append(temp, ignore_index=True)
print()
print(record3)
print()

Training 1: 
Epoch 1/100
299/299 [==============================] - 77s 230ms/step - loss: 0.4543 - accuracy: 0.7852 - val_loss: 0.2949 - val_accuracy: 0.8888
Epoch 2/100
299/299 [==============================] - 65s 218ms/step - loss: 0.1923 - accuracy: 0.9319 - val_loss: 0.3034 - val_accuracy: 0.8841
Epoch 3/100
299/299 [==============================] - 64s 213ms/step - loss: 0.1256 - accuracy: 0.9554 - val_loss: 0.3528 - val_accuracy: 0.8775
Epoch 4/100
299/299 [==============================] - 64s 213ms/step - loss: 0.0919 - accuracy: 0.9661 - val_loss: 0.4094 - val_accuracy: 0.8690
Epoch 5/100
299/299 [==============================] - 63s 212ms/step - loss: 0.0702 - accuracy: 0.9749 - val_loss: 0.4301 - val_accuracy: 0.8718
Epoch 6/100
299/299 [==============================] - 63s 210ms/step - loss: 0.0567 - accuracy: 0.9785 - val_loss: 0.4975 - val_accuracy: 0.8615
Epoch 7/100
299/299 [==============================] - 63s 211ms/step - loss: 0.0504 - accuracy: 0.9789 - val_l

299/299 [==============================] - 64s 214ms/step - loss: 0.0595 - accuracy: 0.9758 - val_loss: 0.4268 - val_accuracy: 0.8699
Epoch 7/100
299/299 [==============================] - 63s 212ms/step - loss: 0.0473 - accuracy: 0.9841 - val_loss: 0.4627 - val_accuracy: 0.8680
Epoch 8/100
299/299 [==============================] - 64s 213ms/step - loss: 0.0438 - accuracy: 0.9824 - val_loss: 0.5203 - val_accuracy: 0.8662
Epoch 9/100
299/299 [==============================] - 64s 215ms/step - loss: 0.0368 - accuracy: 0.9839 - val_loss: 0.5313 - val_accuracy: 0.8662
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 89.72667455673218
Training 7: 
Epoch 1/100
299/299 [==============================] - 70s 205ms/step - loss: 0.4491 - accuracy: 0.7818 - val_loss: 0.3221 - val_accuracy: 0.8802
Epoch 2/100
299/299 [==============================] - 48s 161ms/step - loss: 0.1934 - accuracy: 0.9260 - val_loss: 0.3297 - val_accuracy: 0.8840
Epoch 

## Summary

In [28]:
record3

,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,88.878417,87.747407,87.464654,88.312912,85.956645,89.726675,88.396227,87.641507,88.867927,86.509436,87.950181


In [29]:
report = record3
report = report.to_excel('LSTM_MPQA_3.xlsx', sheet_name='dynamic')